# Which month's DT is applied to a given month's dispensing?

We've always assumed that the November DT would be the basis for all reimbursment in the month of November, but now we're not so sure.

The following SQL computes the median "price per pill" of all tablets and capsules in November 2018.

It then compares it with the drug tariff (or price concession) price of each of those pills.

We show that Category M presentations use the DT price for the month of dispensing, but Category A presentations at least sometimes use the Drug Tariff of the following month.

Correspondence with BSA about this tracked [here](https://github.com/ebmdatalab/openprescribing/issues/1318#issuecomment-453180209)

In [13]:
import pandas as pd

In [3]:
sql = """WITH
  median_price_per_unit AS (
  WITH
    prices_per_unit AS (
    SELECT
      month AS date,
      bnf_code,
      bnf_name,
      IEEE_DIVIDE(net_cost,quantity) AS price_per_unit
    FROM
      ebmdatalab.hscic.normalised_prescribing_standard)
  SELECT
    DISTINCT date,
    bnf_code,
    bnf_name,
    percentile_cont(price_per_unit,
      0.5) OVER (PARTITION BY date, bnf_code) AS median_price_per_unit
  FROM
    prices_per_unit),
  oct_rx AS (
  SELECT
    date,
    bnf_code,
    bnf_name,
    ROUND(median_price_per_unit, 4) AS median_price_per_unit
  FROM
    hscic.vw__median_price_per_unit AS rx
  WHERE
    (rx.bnf_name LIKE '%_Tab%'
      OR rx.bnf_name LIKE '%_Cap%')
    AND rx.date='2018-10-01'
    AND rx.bnf_code NOT LIKE '0410020C0%AC'),
  sept_dt AS (
  SELECT
    dt.bnf_code,
    tariff_category,
    ROUND(COALESCE(dt.concession,
        dt.price_pence)/dt.quantity/100,4) AS dt_price_per_unit
  FROM
    dmd.dt_viewer AS dt
  WHERE
    dt.date='2018-09-01'),
  oct_dt AS (
  SELECT
    dt.bnf_code,
    tariff_category,
    ROUND(COALESCE(dt.concession,
        dt.price_pence)/dt.quantity/100,4) AS dt_price_per_unit
  FROM
    dmd.dt_viewer AS dt
  WHERE
    dt.date='2018-10-01'),
  nov_dt AS (
  SELECT
    dt.bnf_code,
    tariff_category,
    ROUND(COALESCE(dt.concession,
        dt.price_pence)/dt.quantity/100,4) AS dt_price_per_unit
  FROM
    dmd.dt_viewer AS dt
  WHERE
    dt.date='2018-11-01')
SELECT
  oct_rx.bnf_code,
  oct_rx.bnf_name,
  oct_rx.median_price_per_unit,
  sept_dt.tariff_category AS sept_category,
  sept_dt.dt_price_per_unit AS sept_dt_ppu,
  oct_dt.tariff_category AS oct_category,
  oct_dt.dt_price_per_unit AS oct_dt_ppu,
  nov_dt.tariff_category AS nov_category,
  nov_dt.dt_price_per_unit AS nov_dt_ppu
FROM
  oct_rx
INNER JOIN
  sept_dt
ON
  oct_rx.bnf_code = sept_dt.bnf_code
INNER JOIN
  oct_dt
ON
  oct_rx.bnf_code = oct_dt.bnf_code
INNER JOIN
  nov_dt
ON
  oct_rx.bnf_code = nov_dt.bnf_code"""
df = pd.read_gbq(sql, 'ebmdatalab', dialect='standard', verbose=False)

/home/seb/.local/share/virtualenvs/jupyter/local/lib/python3.6/site-packages/pandas/io/gbq.py:108: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  **kwargs)


In [4]:
df = df.set_index('bnf_code')

In [14]:
df.head(1)

,bnf_name,median_price_per_unit,sept_category,sept_dt_ppu,oct_category,oct_dt_ppu,nov_category,nov_dt_ppu
bnf_code,,,,,,,,
0101010C0AAAAAA,Alum Hydrox_Cap 475mg,0.1143,Part VIIIA Category C,0.1143,Part VIIIA Category C,0.1143,Part VIIIA Category C,0.1143


In [8]:
# Exclude items which have more than one price (i.e. more than one VMPP in the DT)
# As these complicate our calculations

In [15]:
df2 = df.join(df.groupby('bnf_code').count()['bnf_name'] > 1, rsuffix='x').reset_index()
df2 = df2[df2['bnf_namex'] == False]

In [16]:
df2.groupby('oct_category').count()

,bnf_code,bnf_name,median_price_per_unit,sept_category,sept_dt_ppu,oct_dt_ppu,nov_category,nov_dt_ppu,bnf_namex
oct_category,,,,,,,,,
Part VIIIA Category A,334,334,334,334,334,334,334,334,334
Part VIIIA Category C,720,720,720,720,720,720,720,720,720
Part VIIIA Category M,489,489,489,489,489,489,489,489,489


In [11]:
# let's disregard anything where the DT never changed
changing = df2[(df2['sept_dt_ppu'] != df2['oct_dt_ppu']) & (df2['nov_dt_ppu'] != df2['oct_dt_ppu'])]
print("There are {} pills which changed price each month during that quarter".format(changing.count()['bnf_code']))

There are 595 pills which changed price each month during that quarter


In [18]:
changing.groupby('oct_category').count()

,bnf_code,bnf_name,median_price_per_unit,sept_category,sept_dt_ppu,oct_dt_ppu,nov_category,nov_dt_ppu,bnf_namex
oct_category,,,,,,,,,
Part VIIIA Category A,128,128,128,128,128,128,128,128,128
Part VIIIA Category M,467,467,467,467,467,467,467,467,467


In [12]:
# items with a chancing price that used the sept DT
changing[changing['median_price_per_unit'] == changing['sept_dt_ppu']]

,bnf_code,bnf_name,median_price_per_unit,sept_category,sept_dt_ppu,oct_category,oct_dt_ppu,nov_category,nov_dt_ppu,bnf_namex
588,040201030AAAQAQ,Risperidone_Orodisper Tab 4mg S/F,1.7954,Part VIIIA Category A,1.7954,Part VIIIA Category A,1.7950,Part VIIIA Category A,1.7954,False
1038,0408010ADAAACAC,Zonisamide_Cap 50mg,0.6005,Part VIIIA Category A,0.6005,Part VIIIA Category A,0.6012,Part VIIIA Category A,0.6005,False
1097,0409010B0AAAAAA,Amantadine HCl_Cap 100mg,0.7311,Part VIIIA Category A,0.7311,Part VIIIA Category A,0.7309,Part VIIIA Category A,0.7311,False
1851,1001010X0AAAAAA,Nabumetone_Tab 500mg,0.1232,Part VIIIA Category A,0.1232,Part VIIIA Category A,0.1234,Part VIIIA Category A,0.1232,False


In [19]:
import numpy as np
conditions = [
    (changing['median_price_per_unit'] == changing['sept_dt_ppu']),
    (changing['median_price_per_unit'] == changing['oct_dt_ppu']),
    (changing['median_price_per_unit'] == changing['nov_dt_ppu'])
]

choices = ['sept', 'oct', 'nov']
changing['dt_used'] = np.select(conditions, choices)

In [20]:
changing.groupby('dt_used').count()

,bnf_code,bnf_name,median_price_per_unit,sept_category,sept_dt_ppu,oct_category,oct_dt_ppu,nov_category,nov_dt_ppu,bnf_namex
dt_used,,,,,,,,,,
0,21,21,21,21,21,21,21,21,21,21
nov,118,118,118,118,118,118,118,118,118,118
oct,452,452,452,452,452,452,452,452,452,452
sept,4,4,4,4,4,4,4,4,4,4


In [23]:
changing[changing['dt_used'] == 'nov'].groupby('oct_category').count()

,bnf_code,bnf_name,median_price_per_unit,sept_category,sept_dt_ppu,oct_dt_ppu,nov_category,nov_dt_ppu,bnf_namex,dt_used
oct_category,,,,,,,,,,
Part VIIIA Category A,118,118,118,118,118,118,118,118,118,118


In [24]:
changing[changing['dt_used'] == 'oct'].groupby('oct_category').count()

,bnf_code,bnf_name,median_price_per_unit,sept_category,sept_dt_ppu,oct_dt_ppu,nov_category,nov_dt_ppu,bnf_namex,dt_used
oct_category,,,,,,,,,,
Part VIIIA Category M,452,452,452,452,452,452,452,452,452,452


In [22]:
changing[changing['dt_used'] == 'sept'].groupby('oct_category').count()

,bnf_code,bnf_name,median_price_per_unit,sept_category,sept_dt_ppu,oct_dt_ppu,nov_category,nov_dt_ppu,bnf_namex,dt_used
oct_category,,,,,,,,,,
Part VIIIA Category A,4,4,4,4,4,4,4,4,4,4


In [110]:
# Missing ones - these look like rounding errors to me
changing[changing['dt_used'] == '0']

,bnf_code,bnf_name,median_price_per_unit,sept_category,sept_dt_ppu,oct_category,oct_dt_ppu,nov_category,nov_dt_ppu,bnf_namex,dt_used
88,0106020M0AAACAC,Senna_Tab 7.5mg,0.0316,Part VIIIA Category M,0.0315,Part VIIIA Category M,0.0317,Part VIIIA Category M,0.0285,False,0
113,0202020L0AABIBI,Furosemide_Tab 500mg,1.3919,Part VIIIA Category A,1.3779,Part VIIIA Category A,1.3882,Part VIIIA Category A,1.3918,False,0
451,0212000B0AAABAB,Atorvastatin_Tab 20mg,0.0297,Part VIIIA Category M,0.0314,Part VIIIA Category M,0.0296,Part VIIIA Category M,0.0279,False,0
614,0402010ABAAAFAF,Quetiapine_Tab 150mg,0.2236,Part VIIIA Category M,0.3900,Part VIIIA Category M,0.2235,Part VIIIA Category M,0.1958,False,0
615,0402010ABAAAKAK,Quetiapine_Tab 300mg,0.2569,Part VIIIA Category M,0.5485,Part VIIIA Category M,0.2568,Part VIIIA Category M,0.2247,False,0
694,0403030E0AAAEAE,Fluoxetine HCl_Cap 60mg,0.2276,Part VIIIA Category M,0.1310,Part VIIIA Category M,0.2277,Part VIIIA Category M,0.2003,False,0
726,0403040X0AAAAAA,Mirtazapine_Tab 30mg,0.0422,Part VIIIA Category M,0.0439,Part VIIIA Category M,0.0421,Part VIIIA Category M,0.0389,False,0
728,0403040X0AAALAL,Mirtazapine_Orodisper Tab 15mg,0.0454,Part VIIIA Category M,0.0413,Part VIIIA Category M,0.0453,Part VIIIA Category M,0.0413,False,0
729,0403040X0AAAMAM,Mirtazapine_Orodisper Tab 45mg,0.0786,Part VIIIA Category M,0.0643,Part VIIIA Category M,0.0787,Part VIIIA Category M,0.0703,False,0
1120,0409010P0AAABAB,Pergolide Mesil_Tab 250mcg,0.3604,Part VIIIA Category A,0.3607,Part VIIIA Category A,0.3605,Part VIIIA Category A,0.3603,False,0


In [ ]:
#Let's take a look at 0501110C0AABHBH Metronidazole_Tab 500mg
# 0501110C0AABHBH	Metronidazole_Tab 500mg	1.8600	Part VIIIA Category A	1.8010	Part VIIIA Category A	1.8281	Part VIIIA Category A	1.8605	False	0
# That could be a rounding error. We have it at 39.06 a pack- 1.8600 per pill
# It was 39.07 in November which is 1.8605